Capture Receipt Images:

In [15]:
import cv2
import numpy as np

# Define the preprocessing function
def preprocess_image(image_path):
    # Read the image from the specified path
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or unable to load.")
    
    # Resize image if it's too large
    max_width = 800  # Set a maximum width for the image
    height, width = image.shape[:2]
    if width > max_width:
        scale_ratio = max_width / width
        new_size = (max_width, int(height * scale_ratio))
        image = cv2.resize(image, new_size)
    
    # Convert to grayscale for clarity
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Enhance contrast and brightness using histogram equalization
    equalized = cv2.equalizeHist(gray)
    
    # Apply adaptive thresholding for sharper text regions
    thresh = cv2.adaptiveThreshold(equalized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY, 11, 2)
    
    # Convert the thresholded image back to the BGR format for Keras-OCR compatibility
    processed_image = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
    
    return processed_image


Implement OCR Processing:

In [16]:

import pytesseract

# Define the OCR function
def extract_tesseract(image):
    # Custom Tesseract configuration
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz,.'
    
    # Extract text with pytesseract
    extracted_text = pytesseract.image_to_string(image, config=custom_config)
    return extracted_text


In [17]:
# Path to the image file
image_path = r"C:\Users\Amina\Downloads\sa4bzhkgewj81.jpg"  # Path to receipt image

# Process the image and extract text
image = preprocess_image(image_path)
text_lines = extract_tesseract(image)

# Print extracted text from receipt in the desired format
print("Extracted Text from Receipt:")
print(text_lines )


Extracted Text from Receipt:
ge
TS ee ae
ee pra oe oe a a
wo bee Toposs,auRe BREAST os a Lo 7 a j
NT UBLIXEEALTVANIE 2100 ee
tee a
ee Y5PAC.BRO TACREANYESS a fae So
4 Presa 281b oe Le ue
Seine abe eee ee ee bee
So REL ee eS
sey py oS CG ee
eS ORGANICcaorS eo a oe
SesavnasHaors, Spe te ee 2
enn ee
es ss ae i ee S 2 So cbean ee
oe sk on oo e
So rand otal e100. ee eee
Sy Soe eS
abled FO gee
ay y Mae cee



In [25]:
import easyocr

def extract_text_easyocr(image):
    reader = easyocr.Reader(['en'], gpu=False)  # Disable GPU if not available
    results = reader.readtext(image, detail=1)  # Use detail=1 to get bounding boxes and text
    
    lines = {}
    
    for result in results:
        # Each result contains (bbox, text, confidence)
        bbox, text, _ = result
        # Get the y-coordinate of the bounding box for grouping lines
        y = int(bbox[0][1])  # Take the y-coordinate of the first corner
        
        if y not in lines:
            lines[y] = []
        lines[y].append(text)
    
    # Sort lines by y-coordinate and join words into a single line
    sorted_lines = sorted(lines.items(), key=lambda x: x[0])
    extracted_lines = [' '.join(line_words) for _, line_words in sorted_lines]
    
    return extracted_lines  # Return list of extracted lines

if __name__ == "__main__":
    image_path = r"C:\Users\Amina\Downloads\sa4bzhkgewj81.jpg"  # Replace with your image path
    preprocessed_image = preprocess_image(image_path)  # Preprocess the image
    extracted_text = extract_text_easyocr(preprocessed_image)  # Extract text
    
    print("Extracted Text from Receipt:")
    for line in extracted_text:
        print(line)



Using CPU. Note: This module is much faster with a GPU.


Extracted Text from Receipt:
PuiE D5 6.00
You) Saved
0.67
ToMAToes
DICED 4
PUB
376
TOMIRASTE
PebwG t4EFHBrE40
09
4
PES GCYHEAaM shrd
3489
PBX E
7459
ThPoSS BURG
2418
CHICK BREAST
BNLSIc
VANIL
PuBLiX FF LT
2a00
4k00
2 FOR:
LMES TRERSIAN
374
0, 59
5499
PAC BROTH CHCKN LS
5375
JIFRRDAFFTECREAMY
GREEN BEANS
189
'PUBLIX 
0,
HZ_TOMATO KETCHUP
6,39
REPPERS GREEN BELI
14 1b1
2549
2484
BELL
PEPPERS RED
055
16
@
3399/
1b
ORGANIC CARROTS
269
BANANA SHALLQIS 
0.2 157
499
IzC
Order Total
Galed Ta
Grand  Tota]
100 C
Credit
Payuent;
100 00
'Change
0


keras_ocr

In [23]:
import keras_ocr

def extract_text_kerasocr(image):
    pipeline = keras_ocr.pipeline.Pipeline()
    images = [image]
    predictions = pipeline.recognize(images)
    
    # Group words by line
    lines = {}
    for text, box in predictions[0]:
        # Get the y-coordinate of the box for grouping lines
        y = int(box[0][1])  # Take the y-coordinate of the first corner
        if y not in lines:
            lines[y] = []
        lines[y].append(text)
    
    # Sort lines by y-coordinate and join words into a single line
    sorted_lines = sorted(lines.items(), key=lambda x: x[0])
    extracted_lines = [' '.join(line_words) for _, line_words in sorted_lines]
    
    return extracted_lines

In [24]:
if __name__ == "__main__":
    image_path = r"C:\Users\Amina\Downloads\sa4bzhkgewj81.jpg"  # Replace with your image path
    preprocessed_image = preprocess_image(image_path)  # Preprocess the image
    extracted_text = extract_text_kerasocr(preprocessed_image)  # Extract text
    
    print("Extracted Text from Receipt:")
    for line in extracted_text:
        print(line)

Looking for C:\Users\Amina\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\Amina\.keras-ocr\crnn_kurapan.h5
5/5 [==============================] - 23s 2s/step
Extracted Text from Receipt:
pud cals o0
6
saved
you
g

q
tomatoes
diced
pub
o
paste
toma
ix
publ
bg
e
bread
wheat
hyg
pe
3
shrd
parm
ency
pox
59
t
burg
imposs
2118
breast
chick
bnls
yanil
st
fe
publix
e
laoor
0o
for
d
l
1e
persian lines

i
l
50
d
g
s
5
chckn
ls
sroth pac
creamy jie
5
ro ft
s
green
puelix beans
c
tomato
hz ketchlp
61 s9
reppers
green
bell
114 jb
z 4j
b
e 204
bell
perpers
red
0f55
tb

8 l5
2019
e
organic
carrots
f
99
l
banana
shalliots
ozo
toso
so
5 6
 40
order
totau
tqo
qg
sa
tax
ls
o q0
grand
tota
to0
q0
credit
fayment
100
oo
change
o q0
